In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_preprocessing.image import img_to_array
import numpy as np
import cv2

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   fill_mode='nearest',
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./custom_dataset/',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 color_mode = 'grayscale')

test_set = test_datagen.flow_from_directory('./test_dataset/',
                                            target_size = (64, 64),
                                            batch_size = 128,
                                            class_mode = 'categorical',
                                            color_mode = 'grayscale')

validation_set = train_datagen.flow_from_directory('./custom_dataset/',
                                                   target_size = (64,64),
                                                   batch_size = 64,
                                                   class_mode = 'categorical',
                                                   color_mode = 'grayscale',
                                                   subset="validation",
                                                   shuffle=True)

Found 7110 images belonging to 6 classes.
Found 308 images belonging to 3 classes.
Found 1775 images belonging to 6 classes.


In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 6, activation = 'softmax'))  # 6 classes for 6 gestures
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 64)        0

In [15]:
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ModelCheckpoint(filepath='saved_models/hand_classifier.h5', save_best_only=True)]

model.fit(training_set,
          epochs = 25,
          validation_data = validation_set,
          callbacks = callbacks)


Epoch 1/25
223/223 [==============================] - 16s 71ms/step - loss: 0.0163 - accuracy: 0.9945 - val_loss: 5.0741e-04 - val_accuracy: 1.0000
Epoch 2/25
223/223 [==============================] - 16s 71ms/step - loss: 0.0192 - accuracy: 0.9937 - val_loss: 0.0018 - val_accuracy: 0.9994
Epoch 3/25
223/223 [==============================] - 17s 76ms/step - loss: 0.0210 - accuracy: 0.9931 - val_loss: 0.0011 - val_accuracy: 0.9994
Epoch 4/25
223/223 [==============================] - 16s 70ms/step - loss: 0.0160 - accuracy: 0.9951 - val_loss: 1.6638e-04 - val_accuracy: 1.0000
Epoch 5/25
223/223 [==============================] - 16s 70ms/step - loss: 0.0120 - accuracy: 0.9972 - val_loss: 7.5435e-04 - val_accuracy: 0.9994
Epoch 6/25
223/223 [==============================] - 16s 72ms/step - loss: 0.0140 - accuracy: 0.9952 - val_loss: 8.3568e-04 - val_accuracy: 0.9994
Epoch 7/25
223/223 [==============================] - 16s 71ms/step - loss: 0.0134 - accuracy: 0.9958 - val_loss: 0.0057

In [16]:
model.save('saved_models/hand_classifier.h5')

In [4]:
actions = {
0:"move_left",
1:"move_right",
2:"click_left",
3:"click_right",
4:"scroll_up",
5:"scroll_down",
}
def get_prediction(img):
    # img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for_pred = cv2.resize(img,(64,64))
    x = img_to_array(for_pred)
    x = x/255.0
    x = x.reshape((1,) + x.shape)
    pred = str(actions[np.argmax(model.predict(x))])
    print(pred)
    return pred


In [5]:
def get_test_prediction(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # if your images are grayscale
    # img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for_pred = cv2.resize(img,(64,64))
    x = img_to_array(for_pred)
    x = x/255.0
    x = x.reshape((1,) + x.shape)
    pred = str(actions[np.argmax(model.predict(x))])
    print(pred)
    return pred

# Test the function
get_test_prediction('test_dataset/0/img801.jpg')
get_test_prediction('test_dataset/1/img575.jpg')
get_test_prediction('test_dataset/2/img739.jpg')

NameError: name 'model' is not defined

In [6]:
import cv2
import pyautogui
import tensorflow as tf

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

# Load the model
model = tf.keras.models.load_model('saved_models/hand_classifier.h5')
aweight = 0.5
num_frames = 0
bg = None

def run_avg(img,aweight):
    global bg
    if bg is None:
        bg = img.copy().astype('float')
        return
    cv2.accumulateWeighted(img,bg,aweight)

def segment(img,thres=25):
    global bg
    diff = cv2.absdiff(bg.astype('uint8'),img)
    _, thresholded = cv2.threshold(diff,thres,255,cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return
    else:
        segmented = max(contours,key = cv2.contourArea)
    return (thresholded,segmented)

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret ==True:
        frame = cv2.flip(frame, 1)
        clone = frame.copy()
        (height, width) = frame.shape[:2]
        roi = frame[100:300, 300:500]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, aweight)
        else:
            hand = segment(gray)

            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (300, 100)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)
                contours, _= cv2.findContours(thresholded,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

                for cnt in contours:
                    if cv2.contourArea(cnt) > 5000:
                        pred = get_prediction(thresholded)

                        if pred == "move_left":
                            pyautogui.moveRel(-50, 0)
                            cv2.putText(clone, pred, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                        elif pred == "move_right":
                            pyautogui.moveRel(50, 0)
                            cv2.putText(clone, pred, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                        elif pred == "click_left":
                            pyautogui.click(button='left')
                            cv2.putText(clone, pred, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                        elif pred == "click_right":
                            pyautogui.click(button='right')
                            cv2.putText(clone, pred, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                        elif pred == "scroll_up":
                            pyautogui.scroll(50)
                            cv2.putText(clone, pred, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
                        elif pred == "scroll_down":
                            pyautogui.scroll(-50)
                            cv2.putText(clone, pred, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.rectangle(clone, (300, 100), (500, 300), (0, 255, 0), 2)
        num_frames += 1
        cv2.putText(clone, "Place your hand in the square to read mouse movements", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('frame', clone)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 94ms/step
click_left
1/1 [==============================] - 0s 17ms/step
scroll_down
1/1 [==============================] - 0s 18ms/step
scroll_down
1/1 [==============================] - 0s 16ms/step
scroll_down
1/1 [==============================] - 0s 17ms/step
scroll_down
1/1 [==============================] - 0s 17ms/step
scroll_down
1/1 [==============================] - 0s 17ms/step
scroll_down
1/1 [==============================] - 0s 16ms/step
scroll_down
1/1 [==============================] - 0s 16ms/step
scroll_down
1/1 [==============================] - 0s 16ms/step
scroll_down
1/1 [==============================] - 0s 17ms/step
scroll_down
1/1 [==============================] - 0s 16ms/step
move_right
1/1 [==============================] - 0s 15ms/step
move_right
1/1 [==============================] - 0s 17ms/step
move_right
1/1 [==============================] - 0s 16ms/step
move_right
1/1 [==============================] - 0s 17m